# Azure AI Search with LlamaIndex

This code demonstrates how to use Azure AI Search with Azure OpenAI and the [LlamaIndex data framework](https://www.llamaindex.ai/).

You need an existing [Azure AI Search service](https://learn.microsoft.com/azure/search/search-create-service-portal), any tier, but it must have capacity and quota for the workload. We recommend Basic or higher for this sample.

You also need [Azure OpenAI](https://learn.microsoft.com/azure/ai-services/openai/how-to/create-resource), with deployments of a text embedding model such as [text-embedding-ada-002](https://learn.microsoft.com/azure/ai-services/openai/concepts/models#embeddings), and a chat model such as [gpt-35-turbo](https://learn.microsoft.com/azure/ai-services/openai/concepts/models#gpt-35).

### Set up a Python virtual environment in Visual Studio Code

1. Open the Command Palette (Ctrl+Shift+P).
1. Search for **Python: Create Environment**.
1. Select **Venv**.
1. Select a Python interpreter. Choose 3.10 or later.

It can take a minute to set up. If you run into problems, see [Python environments in VS Code](https://code.visualstudio.com/docs/python/environments).

### Install packages

In [1]:
! pip install -r azure-search-vector-python-llamaindex-sample-requirements.txt --quiet

### Load .env file

Copy `/code/.env-sample` to an `.env` file in the sample folder, and update accordingly. The search service and Azure OpenAI resource and models must exist, but the search index is created and loaded during code execution. Provide a unique name for the search index. Endpoints, API keys, and Azure OpenAI information can be found in the Azure portal.

In [2]:
from dotenv import load_dotenv
from azure.identity import DefaultAzureCredential
from azure.core.credentials import AzureKeyCredential
import os

load_dotenv(override=True) # take environment variables from .env.

# Make sure your .env file has values for the following environment variables
endpoint = os.environ["AZURE_SEARCH_SERVICE_ENDPOINT"]
credential = AzureKeyCredential(os.environ["AZURE_SEARCH_ADMIN_KEY"]) if len(os.environ["AZURE_SEARCH_ADMIN_KEY"]) > 0 else DefaultAzureCredential()
index_name = os.environ["AZURE_SEARCH_INDEX"]
azure_openai_endpoint = os.environ["AZURE_OPENAI_ENDPOINT"]
# Llama Index does not support RBAC authentication, an API key is required
azure_openai_key = os.environ["AZURE_OPENAI_KEY"]
if len(azure_openai_key) == 0:
    raise Exception("API key required")
azure_openai_embedding_model = os.environ["AZURE_OPENAI_EMBEDDING_MODEL"]
azure_openai_embedding_deployment = os.environ["AZURE_OPENAI_EMBEDDING_DEPLOYMENT"]
azure_openai_chatgpt_deployment = os.environ["AZURE_OPENAI_CHATGPT_DEPLOYMENT"]
azure_openai_api_version = os.environ["AZURE_OPENAI_API_VERSION"]
embedding_dimensions = int(os.getenv("AZURE_OPENAI_EMBEDDING_DIMENSIONS", 1536))

### Configure an embeddings instance

In [3]:
from llama_index.embeddings.azure_openai import AzureOpenAIEmbedding
from llama_index.llms.azure_openai import AzureOpenAI

embeddings = AzureOpenAIEmbedding(
    model_name=azure_openai_embedding_model,
    deployment_name=azure_openai_embedding_deployment,
    api_version=azure_openai_api_version,
    azure_endpoint=azure_openai_endpoint,
    api_key=azure_openai_key
)

llm = AzureOpenAI(
    deployment_name=azure_openai_chatgpt_deployment,
    api_version=azure_openai_api_version,
    azure_endpoint=azure_openai_endpoint,
    api_key=azure_openai_key
)

### Upload, vectorize, and index documents

This step reads PDFs from a local folder, calls the embedding model for vectorization, and then calls a search client to index the content on Azure AI Search.

If you get an exception, such as "type 'Result' is not subscriptable", the problem could be an issue with an outdated langchain package. Run `pip show langchain` from a command prompt to get the version, and `pip uninstall` if the package is old.

In [ ]:
from llama_index.vector_stores.azureaisearch import AzureAISearchVectorStore, IndexManagement, MetadataIndexFieldType
from llama_index.core import StorageContext, VectorStoreIndex, SimpleDirectoryReader
from llama_index.core import Settings
from azure.search.documents.indexes import SearchIndexClient

metadata_fields = {
    "author": "author",
    "theme": ("topic", MetadataIndexFieldType.STRING),
    "director": "director",
}

vector_store = AzureAISearchVectorStore(  
    search_or_index_client=SearchIndexClient(endpoint=endpoint, credential=credential),  
    filterable_metadata_field_keys=metadata_fields,
    index_name=index_name,  
    index_management=IndexManagement.CREATE_IF_NOT_EXISTS,  
    id_field_key="id",  
    chunk_field_key="content",  
    embedding_field_key="content_vector",  
    metadata_string_field_key="metadata",
    doc_id_field_key="doc_id",
    embedding_dimensionality=embedding_dimensions,
    language_analyzer="en.lucene",
    vector_algorithm_type="exhaustiveKnn"
)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
Settings.llm = llm
Settings.embed_model = embeddings
directory = os.path.abspath(os.path.join("..", "..", "..", "..", "data", "benefitdocs"))
documents = SimpleDirectoryReader(directory).load_data()
index = VectorStoreIndex.from_documents(
    documents=documents,
    storage_context=storage_context)


### Perform a vector similarity search

In [5]:
query_engine = index.as_query_engine(llm)
response = query_engine.query("What is included in my Northwind Health Plus plan that is not in standard?")
print(response)

The Northwind Health Plus plan includes coverage for emergency services, mental health and substance abuse treatment, and out-of-network services, which are not included in the Northwind Standard plan.
